In [1]:
import pandas as pd
import numpy as np
import json
import datetime

In [2]:
import os
os.getcwd()

'/home/alex/Solar-irradiance-Team08-IFT6759/Notebooks'

In [3]:
admin_config_path = "../train_cfg.json"
with open(admin_config_path, "r") as fd:
    admin_config = json.load(fd)

In [4]:
dataframe_path = admin_config['dataframe_path']
df = pd.read_pickle('../' + dataframe_path)

In [15]:
df.replace(to_replace="nan", value=np.NaN, inplace=True)
df.head()

,ncdf_path,hdf5_8bit_path,hdf5_8bit_offset,hdf5_16bit_path,hdf5_16bit_offset,BND_DAYTIME,BND_CLEARSKY_GHI,BND_CLOUDINESS,BND_GHI,TBL_DAYTIME,...,GWN_CLOUDINESS,GWN_GHI,PSU_DAYTIME,PSU_CLEARSKY_GHI,PSU_CLOUDINESS,PSU_GHI,SXF_DAYTIME,SXF_CLEARSKY_GHI,SXF_CLOUDINESS,SXF_GHI
iso-datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.500000,0.0,0.0,night,-1.500000,0.0,0.0,night,-3.800000
2010-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.493333,0.0,0.0,night,-1.446667,0.0,0.0,night,-3.893333
2010-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.420000,0.0,0.0,night,-1.766667,0.0,0.0,night,-4.166667
2010-01-01 00:45:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.533333,0.0,0.0,night,-2.033333,0.0,0.0,night,-4.040000
2010-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0,night,-4.1,0.0,...,night,-7.586667,0.0,0.0,night,-2.006667,0.0,0.0,night,-4.133333


In [16]:
station_id = "BND"
image_type = "hdf5_8bit"
daytime = df[station_id + "_DAYTIME"] == 1.0
valid_ghi = ~df[station_id + "_GHI"].isna()
valid_datetime = daytime & valid_ghi
df = df.loc[valid_datetime]

In [19]:
# df.isna().sum()

In [23]:
candidate_datetimes = df.index
candidate_datetimes

DatetimeIndex(['2010-01-01 13:30:00', '2010-01-01 13:45:00',
               '2010-01-01 14:00:00', '2010-01-01 14:15:00',
               '2010-01-01 14:30:00', '2010-01-01 14:45:00',
               '2010-01-01 15:00:00', '2010-01-01 15:15:00',
               '2010-01-01 15:30:00', '2010-01-01 15:45:00',
               ...
               '2015-12-31 20:15:00', '2015-12-31 20:30:00',
               '2015-12-31 20:45:00', '2015-12-31 21:00:00',
               '2015-12-31 21:15:00', '2015-12-31 21:30:00',
               '2015-12-31 21:45:00', '2015-12-31 22:00:00',
               '2015-12-31 22:15:00', '2015-12-31 22:30:00'],
              dtype='datetime64[ns]', name='iso-datetime', length=106385, freq=None)

In [57]:
nb_images_for_training = 3
timedeltas = pd.timedelta_range(start='-15 min', periods=(nb_images_for_training-1), freq='-15 min')
print(candidate_datetimes[2])
print(candidate_datetimes[2] + timedeltas)

2010-01-01 14:00:00
DatetimeIndex(['2010-01-01 13:45:00', '2010-01-01 13:30:00'], dtype='datetime64[ns]', freq='-15T')


In [43]:
all(elem in candidate_datetimes for elem in (candidate_datetimes[2] + timedeltas))

True

In [44]:
retained_datetimes = []
for candidate_datetime in candidate_datetimes:
    if all(elem in candidate_datetimes for elem in (candidate_datetime + timedeltas)):
        retained_datetimes += [candidate_datetime]

In [45]:
print(len(candidate_datetimes))
print(len(retained_datetimes))

106385
101982


In [59]:
# retained_datetimes[:10]

In [ ]:
# TODO: Check if images for the specified datetimes exist on disk

In [8]:
dataframe.columns

Index(['ncdf_path', 'hdf5_8bit_path', 'hdf5_8bit_offset', 'hdf5_16bit_path',
       'hdf5_16bit_offset', 'BND_DAYTIME', 'BND_CLEARSKY_GHI',
       'BND_CLOUDINESS', 'BND_GHI', 'TBL_DAYTIME', 'TBL_CLEARSKY_GHI',
       'TBL_CLOUDINESS', 'TBL_GHI', 'DRA_DAYTIME', 'DRA_CLEARSKY_GHI',
       'DRA_CLOUDINESS', 'DRA_GHI', 'FPK_DAYTIME', 'FPK_CLEARSKY_GHI',
       'FPK_CLOUDINESS', 'FPK_GHI', 'GWN_DAYTIME', 'GWN_CLEARSKY_GHI',
       'GWN_CLOUDINESS', 'GWN_GHI', 'PSU_DAYTIME', 'PSU_CLEARSKY_GHI',
       'PSU_CLOUDINESS', 'PSU_GHI', 'SXF_DAYTIME', 'SXF_CLEARSKY_GHI',
       'SXF_CLOUDINESS', 'SXF_GHI'],
      dtype='object')

In [9]:
target_datetimes = [datetime.datetime.fromisoformat(d) for d in admin_config["target_datetimes"]]

In [13]:
# admin_config

In [14]:
# station = [*stations][0]
station = 'DRA'
ghis = []
cs_ghis = []
for dt in target_datetimes[:]:
    dt += datetime.timedelta(minutes=15)
    cs_ghi = dataframe.lookup([dt],[station + '_CLEARSKY_GHI'])[0]
    cs_ghis += [cs_ghi]
    ghi = dataframe.lookup([dt],[station + '_GHI'])[0]
    ghis += [ghi]
    assert ~np.isnan(ghi)
    assert ~np.isnan(cs_ghi)

In [15]:
len(target_datetimes)

38

In [16]:
np.array([ghis,cs_ghis]).T

array([[-3.90666667e+00,  0.00000000e+00],
       [ 5.46986667e+02,  5.27979586e+02],
       [-4.04000000e+00,  0.00000000e+00],
       [ 5.54073333e+02,  5.37996306e+02],
       [-3.28666667e+00,  0.00000000e+00],
       [ 5.35600000e+02,  5.34871830e+02],
       [-2.64000000e+00,  0.00000000e+00],
       [ 1.94200000e+02,  4.98394927e+02],
       [ 1.11333333e+00,  1.21550426e-01],
       [ 4.25626667e+02,  4.56285026e+02],
       [ 1.53780000e+02,  1.71200203e+02],
       [ 3.45120000e+02,  3.28578942e+02],
       [ 2.36380000e+02,  2.04840773e+02],
       [ 3.59753333e+02,  3.80206857e+02],
       [ 1.87233333e+02,  1.62428131e+02],
       [ 4.05460000e+02,  4.15381167e+02],
       [ 4.49133333e+01,  1.19600476e+02],
       [ 4.23920000e+02,  4.47654850e+02],
       [ 9.50600000e+01,  7.78365262e+01],
       [ 3.21466667e+01,  5.00417728e+02],
       [ 2.46400000e+01,  1.02242402e+01],
       [ 5.18820000e+02,  5.22727054e+02],
       [-4.13333333e-01,  0.00000000e+00],
       [ 5.